In [35]:
import pandas as pd
import numpy as np
import json
from tqdm import tqdm

In [36]:
with open("/research/OFA/results/keyword_extraction/3_task_pretrained/test_predict.json", "r") as outfile:
    predicted_data = json.load(outfile)


In [37]:
true_keywords = predicted_data[2]['ref'].replace(";", " ")
pred_keywords = predicted_data[2]['hyp'].replace(";", " ")
print(true_keywords)
print(pred_keywords)

boj abenomics economics weak yen
japan bank of japan economic conditions and trends


In [38]:
from nltk.stem.snowball import SnowballStemmer as Stemmer
import spacy
nlp = spacy.load("en_core_web_lg")


In [39]:
# print(true_keywords)
# print(nlp(true_keywords))
# print(pred_keywords)
# print(nlp(pred_keywords))
def stem_and_tokenize(keywords):
    tokens = [token.text for token in nlp(keywords)]
    # print(tokens)
    stems = [Stemmer('porter').stem(tok.lower()) for tok in tokens]
    # print(stems)
    return stems
true_tokens = stem_and_tokenize(true_keywords)
pred_tokens = stem_and_tokenize(pred_keywords)


In [40]:
def evaluate(top_N_keyphrases, references, cutoff=10):
    P = len(set(top_N_keyphrases[:cutoff]) & set(references)) / len(top_N_keyphrases[:cutoff])
    R = len(set(top_N_keyphrases[:cutoff]) & set(references)) / len(references)
    F = (2*P*R)/(P+R) if (P+R) > 0 else 0 
    return (P, R, F)

In [22]:
res = evaluate(pred_tokens,true_tokens)
print(res)

(0.125, 0.2, 0.15384615384615385)


In [41]:
scores = []
for i in tqdm(range(0, len(predicted_data))):
    true_keywords = predicted_data[i]['ref'].replace(";", " ")
    pred_keywords = predicted_data[i]['hyp'].replace(";", " ")
    true_tokens = stem_and_tokenize(true_keywords)
    pred_tokens = stem_and_tokenize(pred_keywords)
    result = evaluate(pred_tokens,true_tokens)
    scores.append(result)


P, R, F = np.mean(scores, axis=0)
print(P, R, F)

100%|██████████| 19999/19999 [03:43<00:00, 89.46it/s] 


0.49949277225766026 0.3794409478575097 0.4062643511633774
